# Modular

In [6]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import time
from nltk.corpus import wordnet
import re
import enchant
import random
import numpy as np

# Read data

In [2]:
filename = '/Users/Lyf/OneDrive/study/WISC/2017_spring/Stat_628/hw2/data/train_data.csv'
reviews = pd.read_csv(filename)

# Test number

In [4]:
N = 100000
NN = reviews.shape[0]

In [17]:
np.random.seed(615)
index_ori = np.random.randint(0, NN, N)

In [22]:
yelp = reviews.iloc[index_ori,:].copy()
yelp_ori = yelp.copy()

In [23]:
yelp_ori.shape

(100000, 8)

# 1 加上长度变量

In [24]:
yelp.loc[:, 'text length'] = yelp['text'].apply(len)

In [25]:
d = enchant.Dict("en_US")

# 2 改缩写和换行符

In [26]:
shortcut_dir = {'n\'t': ' not', '\'s': ' is', '\'ve': ' have', '\'d': ' would', '\'m': ' am',
                'Ive': 'I have', 'cannot': 'can not', '\\n': ' '}

In [27]:
print('\\n')

\n


In [28]:
t1 = time.time()
def shortcut_remove(review):
    review = review.lower()
    for i in shortcut_dir:
        review = re.sub(i, shortcut_dir[i], review)
    return review
yelp2 = yelp.copy()
yelp2['text'] = yelp2['text'].apply(shortcut_remove)
time.time() - t1

1.342221736907959

In [30]:
# i=2580
# print(yelp['text'][i])
# print(yelp2['text'][i])

In [33]:
# yelp2['text'][3]

# 3 Delete unEnglish

In [34]:
def check_english(word):
    return(d.check(word))
    
def unEnglish_remove(review):
    review = re.sub('[^a-zA-Z]',' ', review)
    words = pd.Series(review.split())
    index = words.apply(check_english)
    if(len(index)==0):
        return True
    else:
        return(sum(index)/len(index)<0.8)
t1 = time.time()
index = yelp2['text'].apply(unEnglish_remove)
time.time() - t1

262.9892599582672

In [35]:
yelp3 = yelp2[-index].copy()

In [36]:
i=0
yelp2['text'][index][(i*10):(i*10+9)]

20393      l'an passé, j'avais vu les beaux parasols roug...
432049     au pied de cochon foie gras est la "star": une...
418091     i really liked this place, the ambiance, the f...
62467      tolles schwäbisches lokal in denkmalgeschützen...
269774     leider muss ich heute eine schlechte bewertung...
1290509    best. vietnamese. food. that i have ever tried...
500436     my all time favorite at idp chicken tikka masa...
566287     avis à ceux qui souhaite manger tranquille et ...
445361     d'abord, mes étoiles vont pour le bar, pas pou...
Name: text, dtype: object

In [38]:
# yelp2['text'][35]

In [40]:
# yelp2['text'][[24,1167,2580,3750]]

In [41]:
yelp3['text']

965810     the ambiance was nice, a typical japanese rest...
153400     this was my first time here and i must say i r...
1338271    always come here for their chir dashi donburri...
1234172    i went out to matadors with my friend sara on ...
224448     solid 4 star mexican food at very reasonable p...
983577     tonight i ordered the chicken pho, and i reall...
551258     this is a very convenient place for a jimmy jo...
1065423    loved it!  i am so happy they are finally open...
1528164    the food and ambiance was great. i came for br...
551823     the line was long but friendly wholesome patie...
1471410    this place is grossly overpriced.  not even wo...
376886     the curry crepe was slap your mama good! perfe...
1523597    i went to dunkin donuts one night and they had...
693841     great service. fun atmosphere. amazing food fo...
799721     finally was able to make it here for breakfast...
1356334    tried this mexican food spot for the first tim...
166387     my wife and i

# Get positive and negative

In [42]:
# reviews_total = ''
# for i in range(N):
#     reviews_total = reviews_total+" "+yelp.loc[i, 'text']

In [43]:
# reviews_total
# positive_words = ["nice","prime","great","awesome","yummy","friendly","delicious",
#                   "love","spicy","good","favorite","well","clean","enjoy"]
# negetive_words = ["suck","terrible","amazing","messy","worse","hit","miss"]

In [44]:
# print(reviews['text'][9])

# 4 Add not

In [45]:
inverter = ['not', 'never', 'lack', 'no']
def add_not(sentence):
    words = sentence.split().copy()
    not_index = 0
    for index, word in enumerate(words):
        if(not_index != 0):
            words[index] = 'not' + words[index]
        if(re.match('.*[,.?!\'\"]$', word)):
            not_index = 0
        if(word in inverter):
            not_index = 1
    #         print(index)
    sentence = " ".join(words)
    return(sentence)
t1 = time.time()
yelp4 = yelp3.copy()
yelp4['text'] = yelp3['text'].apply(add_not)
time.time() - t1

13.161549806594849

In [47]:
# i=0
# print(yelp3['text'][i])
# print(yelp4['text'][i])

# 5 去标点

In [48]:
# def punctuation_remove(review):
#     review = review.lower()
#     review = re.sub("[^a-zA-Z_]", " ", review)
#     return review
# t1 = time.time()
# yelp5 = yelp4.copy()
# yelp5['text'] = yelp4['text'].apply(punctuation_remove)
# time.time() - t1

In [34]:
# filename_chen = "../data/chen_first"+str(N)+".csv"
# yelp4.to_csv(filename_chen, index = False)

In [49]:
# yelp5['text'][0]

# 5 去时态

In [36]:
# from nltk.stem.wordnet import WordNetLemmatizer 
# lem = WordNetLemmatizer()

# from nltk.stem.porter import PorterStemmer 
# stem = PorterStemmer()
# def tense_remove(review):
#     review = review.split()
#     return " ".join([stem.stem(w) for w in review])
# t1 = time.time()
# yelp5 = yelp4.copy()
# yelp5['text'] = yelp5['text'].apply(tense_remove)
# time.time() - t1

# 7 去停词

In [37]:
# stops = set(stopwords.words("english"))
# stops2 = list(stops)
# stops22 = ['not_'+i for i in stops2]
# stop_all = stops2+stops22

In [38]:
# def noise_remove(review):
#     review = review.split()
#     useful_review = [w for w in review if not w in stop_all]  
#     return " ".join([w for w in useful_review])
# t1 = time.time()
# yelp7 = yelp6.copy()
# yelp7['text'] = yelp7['text'].apply(noise_remove)
# time.time() - t1

In [50]:
filename_li = "../../data/random"+str(N)+".csv"

In [51]:
filename_li

'../../data/random100000.csv'

In [52]:
yelp4.to_csv(filename_li, index = False)

In [16]:
filename_shi = "../data/shi_first"+str(N)+"withoutNot.csv"
yelp7.to_csv(filename_shi, index = False)

In [20]:
filename_li = "../data/li_first"+str(N)+"_2.csv"
yelp6.to_csv(filename_li, index = False)

In [38]:
yelp5

,stars,name,text,date,city,longitude,latitude,categories,text length
0,1,McDonald's,seriously cannot stand this mcdonald is they ...,2014-12-29,Glendale,-112.205020,33.509597,"['Burgers', 'Fast Food', 'Restaurants']",828
1,5,Tom Colicchio's Craftsteak,amazing food truly excellent best lobster bis...,2013-03-07,Las Vegas,-115.169751,36.102918,"['Steakhouses', 'Restaurants', 'Cheesesteaks',...",209
2,5,Fishman Lobster Clubhouse Restaurant,this was my second time here and the seafood ...,2015-11-24,Toronto,-79.300795,43.824234,"['Seafood', 'Restaurants', 'Chinese', 'Live/Ra...",1234
3,1,Bonjour Brioche,long story short bunch of rude heartless an...,2016-12-20,Toronto,-79.346287,43.659795,"['Breakfast & Brunch', 'French', 'Restaurants']",1536
4,4,Dilly's Deli,we grabbed some dinner here last night before ...,2010-09-28,Tempe,-111.945365,33.422175,"['Caterers', 'Sandwiches', 'Event Planning & S...",416
5,5,Angry Crab Shack,great place came in for lunch brodey ashley...,2016-04-07,Peoria,-112.222889,33.596701,"['Seafood', 'American (Traditional)', 'Cajun/C...",404
6,1,The Good Egg,me and wife have been going to the good egg si...,2016-03-08,Glendale,-112.140126,33.714283,"['Restaurants', 'Breakfast & Brunch']",562
7,4,Blueberry Hill Family Restaurant,huge menu selection the restaurant looks grea...,2015-10-15,Las Vegas,-115.207537,36.114985,"['Nightlife', 'Breakfast & Brunch', 'American ...",291
8,3,Zoozacrackers Deli,what do you do if it is getting close to midni...,2010-11-01,Las Vegas,-115.165365,36.126595,"['Delis', 'Restaurants', 'Sandwiches', 'Burgers']",513
9,3,Anjappar Chettinad Restaurants,i saw this restaurant on a whim when i was loo...,2016-05-17,Mississauga,-79.641966,43.599596,"['Indian', 'Buffets', 'Halal', 'Restaurants']",1075
